# Linux内核对设备的分类

> linux的文件种类：

    1. -：普通文件

    2. d：目录文件

    3. p：管道文件

    4. s：本地socket文件

    5. l：链接文件

    6. c：字符设备

    7. b：块设备

ls -l 可以查看文件的类型

drwxrwxr-x 

-rw-rw-r--

每个文件都有文件名、元信息、文件内容。这7种文件都有这三个部分。只是他们的文件内容不同。元文件内有用户信息用户组信息创建时间什么的。他用一个inode结构体对象来存。

再外存中没有管道文件的文件内容。本地socket文件也相当于没有文件内容，他只是本地客户端和服务器的地址值；类似于网络socket的ip+端口号。软连接的内容是它指向文件的路径，硬链接是别名。字符设备和块设备只是提供了一个文件名，告诉我们操作哪个文件相当于操作这些设备。

> Linux内核按驱动程序实现模型框架的不同，将设备分为三类：

1. 字符设备：按字节流形式进行数据读写的设备，一般情况下按顺序访问，数据量不大，一般不设缓存（不部分情况顺序访问，当然也可随机访问）

2. 块设备：按整块进行数据读写的设备，最小的块大小为512字节（一个扇区），块的大小必须是扇区的整数倍，Linux系统的块大小一般为4096字节（4K），随机访问，设缓存以提高效率

（块设备一般都会套上一个文件系统，机械硬盘就是一圈一圈的磁道。磁道间被分成多个扇区）

3. 网络设备：针对网络数据收发的设备

（网卡、蓝牙都属于网络设备）

（所以一切皆文件是要打引号的因为网络设备就不是文件而是网卡）

> 总体框架图：

![alt text](image.png)

左面这俩就是open、close那套接口,右面就是socket那套接口 

# 字符设备驱动框架 

> 作为字符设备驱动的要素  

1. 必须有一个设备号，用在众多的设备驱动中理行区分  
   
2. 用户必须知道设备驱动对应的设备节点（设备文件）
      
3. 对设备操作其实就是对文件操作
   
   应用空间操作open,read,write的时候实际在驱动代码有对应的open,read,write函数  




   /dev/lcd ,相当于文件的名字
  
   cat /pro/devices 查看当前系统中的设备

设备节点用于对不同类型的设备加以区分

> 创建设备节点

1. 手动创建

使用命令

如果已知一个设备的主次设备号，应用层指定好设备文件名，那么可以用mknod命令在/dev目录创建代表这个设备的文件，即此后应用程序对此文件的操作就是对其代表的设备操作，mknod用法如下：

``` kernel
   cd /dev
     
   mknod 设备文件名 设备种类(c为字符设备,b为块设备)  主设备号  次设备号    //ubuntu下需加sudo执行
```

mknode /dev/设备名 类型 主设备号 次设备号
        
比如：
   mknode /dev/chr0 c 250 0
            
缺点： /dev/目录中的文件都是在内存中，断电后/dev/文件会消失
   
2. 自动创建（通过udev/mdev机制）

>> 首先创建一个类：

```c
    struct class *class_create(owner, name) // 创建一个类
```
   
   参数1: THIS_MODULE
   
   参数2：字符串名字，自定义
   
   返回一个class指针
    
>> 接着创建设备文件    

```C
   //创建一个设备文件
   struct device *device_create(struct class *class, struct device *parenr, dev_t devt, void *drvdata, const char *fmt, ...)
```

   参数1: class 结构体， class_create调用之后的返回值

   参数2: 表示父亲，一般填NULL

   参数3: 设备号类型dev_t

         dev_t devt
         #define MAJOR(dev) ((unsigned int) ((dev) >> MINORBITS))
         #define MINOR(dev) ((unsigned int)((dev) & MINORMASK))
         # define MKDEV(ma,mi) (((ma) << MINORBITS)| (mi))

    参数4: 私有数据， 一般直接填NULL
    参数5和参数6: 可变参数，字符串，表示设备节点的名字

1. 销毁设备
   参数1： class 结构体，class_create调用之后的返回值
   参数2: 设备号类型 dev_t 
   
3. 在驱动中实现文件io的接口，应用程序可以调用文件io
   1. 在驱动中实现文件io操作接口： struct file_operation 
    函数指针的集合，其实就是接口，我们写驱动的时候需要去实现
   2. 应用程序如何去调用文件io去控制驱动 open read write
4. 应用程序需要传递数据给驱动
   int copy_to_user(void *to, const void *from, unsigned long count);
   将数据从内核空间拷贝到用户空间,一般是在驱动中chr_drv_read()用
   参数1: 内核驱动中的一个buffer
   参数2: 应用空间的一个buffer
   参数3: 个数
   返回值大于0表示出错，剩下多少个没有拷贝成功；等于0，表示正确
   int copy_from_user(void *to, const void *from, unsigned long count);
   将数据从用户空间拷贝到内核空间,一般是在驱动中chr_drv_write()用
5. 控制外设，其实主湜控制地址，内核驱动中是通过虚拟地址操作
   将物理地址映射成为虚拟地址
   void *ioremap(cookie, size)
   参数1: 物理地址
   参数2: 长度
   去映射——解除映射
   void iounmap(void __iomem *addr)
   参数1: 映射之后的虚拟地址
6. 通过驱动控制LED
   led --- GPX2_7 --- GPX2CON = 0x11000C40
                      GPX2DAT = 0x11000C44
   将0x11000C40 映射成虚拟地址
   对虚拟地址中的[32:28] = 0x1
7. 应用程序和驱动扮演的是什么角色
   
   用户态 ： 应用程序
   
      玩策略： 怎么去做
         对LED：
         1. 一闪一闪
         2. 一直亮
         3. 10s闪一次
         4. ....

   ——————————————————
   
   内核态 ： 驱动

      玩机制 ： 能做什么 
      对led ： 亮 和 灭
8. 编写字符设备驱动的步骤和规范
   步骤：
      1. 实现模块的加载和卸载的入口函数
      2. 在模块加载入口函数中
         1. 申请主设备号（用于区分和管理不同的字符设备）
         2. 创建设备节点文件（为用户提供一个可操作的文件接口--open()）
         3. 硬件的初始化
            1. 地址的映射
            2. 中断的申请
            3. 实现硬件寄存器的初始化
         4. 实现file_operations
9.  操作寄存器地址的方式：
    1.  volatile unsigned long *gpxcon;
         *gpxcom &= ~(0xf<<20)
    2.  readl/writel();
        1.  u32 readl(const volatile void __iomem *addr) //从地址中读取地址空间到值
        2.  writel(unsigned long value, const volatile void __iomem *addr)  //将value 的值写入到addr地址


In [ ]:
//chr_drv.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/fs.h>  // register_chrdev
#include <linux/device.h>  //device_create
#include <asm/uaccess.h>  //copy_to_user

#define GPX2_CON 0x11000C40
#define GPX2_DAT 0x11000C44
//存放虚拟地址的指针
volatile unsigned int *gpx2conf;
volatile unsigned int *gpx2data;
//静态指定方式
static unsigned int dev_major = 250;

static struct class *dvcls;
static struct device *dev;

static int kernel_val = 555;

ssize_t chr_drv_read(struct file *file, char __user *buf, size_t count, loff_t *ppos)
{
    prink("-----%s-----\n", __FUCNTION__);
    int ret;
    ret = copy_to_user(buf, &kernel_val, count);
    if(ret > 0)
    {
        printk("copy_to_user failed\n");
        return -EFAULT;
    }
    return 0;
}
ssize_t chr_drv_write(struct file *file, const char __user *buf, size_t count, loff_t *ppos)
{
    prink("-----%s-----\n", __FUCNTION__);
    int ret;
    int value;

    ret = copy_from_user(&value, buf, count);
    if(ret > 0)
    {
        printk("copy_from_user failed\n");
        return -EFAULT;
    }
    
    //打印value的值 
    //printk("__KERN__ : value = %d\n", value)；
    
    if(value == 0)
    {
        *gpx2data &= ~(0x1 << 7);
    }
    else
    {
        *gpx2data |= (0x1 << 7);
    }
    return 0;
}
int chr_drv_open(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
int chr_drv_close(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
static struct my_fops = {
    .open = chr_drv_open,
    .read = chr_drv_read,
    .write = chr_drv_write,
    .release = chr_drv_close,
};
static int chr_dev_init(void)
{
    //模块装载，一般都是申请设备资源
/***************************申请设备号*********************************/
    int ret;

    ret = register_chrdev(dev_major, "chr_dev_test", &my_fops);
    if (ret == 0)
    {
        printk("register ok\n");
    }
    else
    {
        printk("register failed\n");
        return -EINVAL;
    }
    return 0;
/**********************创建设备节点***************************************/
    devcls = class_create(THIS_MODULE, "chr_dev_test");
    dev = device_create(devcls, NULL, MKDEV(dev_major, 0), NULL, "chr2");

/***********************对地址进行映射***********************************************/    
    gpx2conf = ioremap(GPX2_CON, GPX2_SIZE);
    gpx2data = gpx2conf + 1;
/***********************需要配置gpio功能为输出***********************************************/

    *gpx2conf &= ~(0xf << 28);
    *gpx2conf |= (0x1 << 28);
}

static void chr_dev_exit(void)
{
    device_destroy(devcls, MKDEV(dev_major, 0));
    class_destroy(devcls);
    unregister_chrdev(dev_major, "chr_dev_test");
}
module_init(chr_dev_init);
module_exit(chr_dev_exit);
MODULE_LICENSE("GPL");

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/stat.h>


int main(int argc, char *argv[])
{
    //调用驱动
    int fd;
    int value = 0;

    fd = open("/dev/chr2", O_RDWR);
    if (fd < 0)
    {
        perror("open");
        exit(1);
    }
    printf("__USER__ : value = %d\n", value);
    //重新给value赋值
    value = 666;
    
    read(fd, &value, 4);
    write(fd, &value, 4);
    close(fd);
    return 0;
}

In [ ]:
//chr_drv.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/fs.h>  // register_chrdev
#include <linux/device.h>  //device_create
#include <asm/uaccess.h>  //copy_to_user
#include <asm/io.h>  //ioremap

#define GPX2_CON 0x11000C40
#define GPX2_DAT 0x11000C44
//存放虚拟地址的指针
volatile unsigned int *gpx2conf;
volatile unsigned int *gpx2data;
//静态指定方式
static unsigned int dev_major = 250;

static struct class *dvcls;
static struct device *dev;

static int kernel_val = 555;

ssize_t chr_drv_read(struct file *file, char __user *buf, size_t count, loff_t *ppos)
{
    prink("-----%s-----\n", __FUCNTION__);
    int ret;
    ret = copy_to_user(buf, &kernel_val, count);
    if(ret > 0)
    {
        printk("copy_to_user failed\n");
        return -EFAULT;
    }
    return 0;
}
ssize_t chr_drv_write(struct file *file, const char __user *buf, size_t count, loff_t *ppos)
{
    int ret;
    int value;

    ret = copy_from_user(&value, buf, count);
    if(ret > 0)
    {
        printk("copy_from_user failed\n");
        return -EFAULT;
    }
      
    if(value == 0)
    {
        *gpx2data &= ~(0x1 << 7);
    }
    else
    {
        *gpx2data |= (0x1 << 7);
    }
    return 0;
}
int chr_drv_open(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
int chr_drv_close(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
static struct my_fops = {
    .open = chr_drv_open,
    .read = chr_drv_read,
    .write = chr_drv_write,
    .release = chr_drv_close,
};
static int chr_dev_init(void)
{
    //模块装载，一般都是申请设备资源
/***************************申请设备号*********************************/
    int ret;

    ret = register_chrdev(dev_major, "chr_dev_test", &my_fops);
    if (ret == 0)
    {
        printk("register ok\n");
    }
    else
    {
        printk("register failed\n");
        return -EINVAL;
    }
    return 0;
/**********************创建设备节点***************************************/
    devcls = class_create(THIS_MODULE, "chr_dev_test");
    dev = device_create(devcls, NULL, MKDEV(dev_major, 0), NULL, "chr2");

/***********************对地址进行映射***********************************************/    
    gpx2conf = ioremap(GPX2_CON, GPX2_SIZE);
    gpx2data = gpx2conf + 1;
/***********************需要配置gpio功能为输出***********************************************/

    *gpx2conf &= ~(0xf << 28);
    *gpx2conf |= (0x1 << 28);
    return 0;
}

static void chr_dev_exit(void)
{
    device_destroy(devcls, MKDEV(dev_major, 0));
    class_destroy(devcls);
    unregister_chrdev(dev_major, "chr_dev_test");
}
module_init(chr_dev_init);
module_exit(chr_dev_exit);
MODULE_LICENSE("GPL");

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/stat.h>


int main(int argc, char *argv[])
{
    //调用驱动
    int fd;
    int value = 0;

    fd = open("/dev/chr2", O_RDWR);
    if (fd < 0)
    {
        perror("open");
        exit(1);
    }
    printf("__USER__ : value = %d\n", value);
    //应用程序去控制灯的亮灭
    while(1)
    {
        value = 0;
        write(fd, &value, 4);
        sleep(1);
        value = 1;
        write(fd, &value, 4);
        sleep(1);
    }
    
    close(fd);
    return 0;
}

对代码进行完善

规范：

1. 面向对象编程
   
   用一个结构体来表示一个对象

2. 做出错处理
   在某个位置出错了，要将之前申请到的资源进行释放

`register_chrdev`函数是Linux内核中的一个函数，用于注册一个字符设备。它的原型如下：



In [ ]:
int register_chrdev(unsigned int major, const char *name, const struct file_operations *fops);



该函数接受三个参数：

1. `major`：一个无符号整数，表示字符设备的主设备号。如果这个参数为0，那么内核将会动态分配一个主设备号。
2. `name`：一个字符串，表示字符设备的名称。这个名称将会出现在`/proc/devices`文件中。
3. `fops`：一个指向`file_operations`结构体的指针，表示字符设备的文件操作函数集合。

`register_chrdev`函数的作用是注册一个字符设备，使得这个字符设备可以被系统识别和使用。

以下是一个示例，演示如何使用`register_chrdev`函数注册一个字符设备：



In [ ]:
#include <linux/fs.h>

unsigned int major = 0;
const char *name = "mychar";
struct file_operations my_fops = {
    // 初始化你的文件操作函数集合
};

int ret = register_chrdev(major, name, &my_fops);
if (ret < 0) {
    printk(KERN_ERR "Failed to register character device\n");
    return ret;
}



在上面的示例中，我们首先定义了一个无符号整数`major`，一个字符串`name`，和一个`file_operations`结构体`my_fops`。然后，我们调用`register_chrdev`函数注册一个字符设备。

请注意，如果`register_chrdev`函数返回一个负数，那么表示注册失败。你应该检查返回值，并处理错误。

另外，当你不再需要这个字符设备时，你应该使用`unregister_chrdev`函数来注销它，例如：



In [ ]:
unregister_chrdev(major, name);



然而，`register_chrdev`和`unregister_chrdev`函数在现代的Linux内核中已经被废弃，建议使用`cdev_init`、`cdev_add`和`cdev_del`函数来代替。

In [ ]:
//led_drv.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/fs.h>  // register_leddev
#include <linux/device.h>  //device_create
#include <asm/uaccess.h>  //copy_to_user
#include <asm/io.h>  //ioremap
#include <linux/slab.h>  //kmalloc

//设计一个类型，描述一个设备的信息
struct led_desc
{
    unsigned int dev_major; //主设备号
    struct class *cls;  //类指针
    struct device *dev;  //创建设备文件
    void *reg_virt_base;  //虚拟地址
}

struct led_desc *led_dev; //声明对象，用于全局共享

static int kernel_val = 555;

ssize_t led_drv_read(struct file *file, char __user *buf, size_t count, loff_t *ppos)
{
    prink("-----%s-----\n", __FUCNTION__);
    int ret;
    ret = copy_to_user(buf, &kernel_val, count);
    if(ret > 0)
    {
        printk("copy_to_user failed\n");
        return -EFAULT;
    }
    return 0;
}
ssize_t led_drv_write(struct file *file, const char __user *buf, size_t count, loff_t *ppos)
{
    int ret;
    int value;

    ret = copy_from_user(&value, buf, count);
    if(ret > 0)
    {
        printk("copy_from_user failed\n");
        return -EFAULT;
    }
      
    if(value)
    {
        writel(readl(led_dev->reg_virt_base + 4) | (1 << 7), led_dev->reg_virt_base + 4);
    }
    else
    {
        writel(readl(led_dev->reg_virt_base + 4) & ~(1 << 7), led_dev->reg_virt_base + 4);
    }
    return 0;
}
int led_drv_open(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
int led_drv_close(struct inode *inode, struct file *file)
{
    prink("-----%s-----\n", __FUCNTION__);
    return 0;
}
static struct my_fops = {
    .open = led_drv_open,
    .read = led_drv_read,
    .write = led_drv_write,
    .release = led_drv_close,
};
static int led_dev_init(void)
{
    //0, 实例化全局的设备对象--分配空间
    // GFP_KERNEL, 如果当前内存不够，会阻塞
    led_dev = kmalloc(sizeof(struct led_desc), GFP_KERNEL);  //分配空间 
    if(led_dev == NULL)
    {
        printk(KERN_ERR "kmalloc failed\n");
        return -ENOMEM;
    }

    //模块装载，一般都是申请设备资源
/***************************申请设备号*********************************/
    /*静态指定  
    led_dev->dev_major = 250; 
    int ret;
    ret = register_chardev(led_dev->dev_major, "led_dev_test", &my_fops);
    
    if (ret == 0)
    {
        printk("register ok\n");
    }
    else
    {
        printk("register failed\n");
        return -EINVAL;
    }
    */
    /*动态分配*/
    led_dev->dev_major = register_chrdev(0, "led_dev_test", &my_fops);
    if (led_dev -> dev_major < 0)
    {
        printk(KERN_ERR "register failed\n");
        ret = -ENODEV;
        goto err_0; 
    }
/**********************创建设备节点***************************************/
    led_dev->cls = class_create(THIS_MODULE, "led_dev_test");
    if(IS_ERR(led_dev -> cls))
    {
        printk(KERN_ERR "class_create error\n");
        ret = PTR_ERR(led_dev->cls);//将指针出错的具体原因转换成一个出错码
        goto err_1;
    }
    led_dev->dev = device_create(led_dev->cls, NULL, MKDEV(led_dev->dev_major, 0), NULL, "led%d", 0);
    if(IS_ERR(led_dev -> dev))
    {
        printk(KERN_ERR "device_create error\n");
        ret = PTR_ERR(led_dev->dev);//将指针出错的具体原因转换成一个出错码
        goto err_2;
    }


/***********************对地址进行映射***********************************************/    
    led_dev->reg_virt_base = ioremap(GPX2_CON, GPX2_SIZE);
    if(led_dev->reg_virt_base==NULL)
    {
        printk(KERN_ERR "ioremap error\n");
        ret = -ENOMEM;
        goto err_3;
    }
/***********************配置gpio功能为输出***********************************************/
    u32 value = readl(led_dev->reg_virt_base);
    value &= ~(0xf << 28);
    value |= (0x1 << 28);
    writel(value, led_dev->reg_virt_base);
    
//递进执行下面标签中的语句，来逐级释放
err_3:
    device_destory(led_dev->cls, MKDEV(led_dev->dev_major,0));
err_2:
    class_destory(led_dev->cls);
err_1:
    unregister_chrdev(led_dev->dev_major, const char *name);
err_0:
    kfree(led_dev);
}


static void led_dev_exit(void)
{
    iounmap(led_dev->reg_virt_base);
    device_destory(led_dev->cls, MKDEV(led_dev->dev_major,0));
    class_destory(led_dev->cls);
    unregister_chrdev(led_dev->dev_major, const char *name);
    kfree(led_dev);
}
module_init(led_dev_init);
module_exit(led_dev_exit);
MODULE_LICENSE("GPL");

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/stat.h>


int main(int argc, char *argv[])
{
    //调用驱动
    int fd;
    int value = 0;

    fd = open("/dev/led0", O_RDWR);
    if (fd < 0)
    {
        perror("open");
        exit(1);
    }
    printf("__USER__ : value = %d\n", value);
    //应用程序去控制灯的亮灭
    while(1)
    {
        value = 0;
        write(fd, &value, 4);
        sleep(1);
        value = 1;
        write(fd, &value, 4);
        sleep(1);
    }
    
    close(fd);
    return 0;
}